In [ ]:
import numpy as np
import qutip
import dimer
from pickle import load
import matplotlib.pyplot as plt

plt.style.use(["./config/stylelib/thesis.mplstyle"])

In [ ]:
lmbd_1_list = [0, .7, 1.1, 2/np.sqrt(3), 1.5, 3]
lmbd_2_list = [0, .7, 1.1, 2/np.sqrt(3), 3., 3.2]
ntraj_list= [10**2]
omega_S_list = [1]

T_list = [100]
dt_eval_list = [T/2/100 for T in T_list]
t_eval_list = [np.arange(T/2, T+dt_eval, dt_eval)
               for T, dt_eval in zip(T_list, dt_eval_list)]

solver = "mcsolve"
# solver = "mcsolve"
if solver == "mcsolve":
    dt_list = [None]
elif solver == "trsolve":
    dt_list = [0.01]

no_click = False

psi0_list = [qutip.tensor(qutip.basis(2, 1), qutip.basis(2, 1))]

sim_list = dimer.get_sim_list(lmbd_1_list,
                              lmbd_2_list,
                              omega_S_list,
                              psi0_list,
                              t_eval_list,
                              ntraj_list,
                              dt_list,
                              solver,
                              no_click)

In [ ]:
data = []
for sp in sim_list:
    f = open(f"data/{str(sp)}.pkl", "rb")
    data.append(load(f))
    f.close()

In [ ]:
N_bins = 36
rs = []
thetas = []
for i, sim in enumerate(sim_list):
    states = np.array(data[i][1].states, dtype=object).flatten()
    r = np.empty(len(states))
    t = np.empty(len(states))
    for j, psi in enumerate(states):
        pstate = psi.ptrace(0)
        r[j], t[j] = dimer.pstate_to_yz_bloch(pstate)
    rs.append(r)
    thetas.append(t)

In [ ]:
hist_range = (-np.pi, np.pi)

histograms = []
for i, sim in enumerate(sim_list):
    histograms.append(np.histogram(thetas[i], N_bins, range=hist_range, density=True))

In [ ]:
for i, sim in enumerate(sim_list):
    plt.figure()

    # plt.hist(thetas[i], bins=N_bins, range=hist_range, density=True)

    freqs, bin_edges = histograms[i]
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.plot(bin_centers, freqs, ".")

    plt.title(rf"$\lambda_1 = {sim.lmbd_1}$, $\lambda_2 = {sim.lmbd_2}$")